In [9]:
import os
import re
from pathlib import Path


def find_api_endpoints(python_dir):
    """Find all API endpoints in Python files"""
    endpoints = {}
    patterns = [
        r'@(?:app|router)\.(get|post|put|delete|patch)\(["\']([^"\']+)["\']',
    ]

    for root, _, files in os.walk(python_dir):
        for file in files:
            if file.endswith('.py'):
                file_path = os.path.join(root, file)
                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        content = f.read()
                        for pattern in patterns:
                            matches = re.finditer(pattern, content)
                            for match in matches:
                                method = match.group(1)
                                path = match.group(2)
                                pattern_str = re.escape(path).replace(r'\{[^}]+}', r'[^/]+')
                                endpoints[path] = {
                                    'method': method,
                                    'file': file_path,
                                    'used': False,
                                    'pattern': re.compile(pattern_str),
                                    'core_path': path.strip('/')  # Store path without leading/trailing slashes
                                }
                except (IOError, UnicodeDecodeError) as e:
                    print(f"Warning: Could not read {file_path}: {e}")
    return endpoints


def search_in_ts_files(ts_dir, endpoints):
    """Search for endpoint usage in TypeScript files"""
    for root, _, files in os.walk(ts_dir):
        for file in files:
            if file.endswith(('.ts', '.svelte')):
                file_path = os.path.join(root, file)
                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        content = f.read()
                        for path, info in endpoints.items():
                            core_path = info['core_path']
                            # Extract the significant part of the path (after first parameter)
                            significant_path = re.sub(r'^[^/]+/', '', core_path)  # Remove anything before first param
                            if not significant_path:
                                significant_path = core_path  # Fallback to full path if no prefix

                            # Check multiple ways the endpoint might be used
                            if info['pattern'].search(content):
                                info['used'] = True
                            # Check template literal with any prefix plus significant path
                            elif re.search(f'`[^`]*{re.escape(significant_path)}[^`]*`', content):
                                info['used'] = True
                            # Check direct string usage with common prefixes
                            elif any(f'{quote}{prefix}{significant_path}{quote}' in content
                                     for quote in ['"', "'"]
                                     for prefix in ['', '/', 'users/', 'api/', '/users/', '/api/',
                                                    'functions/', '/functions/']):
                                info['used'] = True
                except (IOError, UnicodeDecodeError) as e:
                    print(f"Warning: Could not read {file_path}: {e}")


def main():
    # Configure these paths according to your project structure
    python_dir = './backend'  # Path to your Python backend directory
    ts_dir = './frontend'     # Path to your TypeScript/Svelte frontend directory

    # Find all API endpoints
    endpoints = find_api_endpoints(python_dir)

    # Search for their usage in TS files
    search_in_ts_files(ts_dir, endpoints)

    # Print results
    print("\nAPI Endpoints Analysis:")
    print("=====================")

    unused_endpoints = []
    used_endpoints = []

    for path, info in endpoints.items():
        if info['used']:
            used_endpoints.append((path, info))
        else:
            unused_endpoints.append((path, info))

    if used_endpoints:
        print("\nUsed Endpoints:")
        for path, info in sorted(used_endpoints):
            print(f"✓ {info['method'].upper()} {path} (in {info['file']})")

    if unused_endpoints:
        print("\nUnused Endpoints (potential candidates for removal):")
        for path, info in sorted(unused_endpoints):
            print(f"✗ {info['method'].upper()} {path} (in {info['file']})")
    else:
        print("\nNo unused endpoints found!")

    print(f"\nSummary:")
    print(f"Total endpoints: {len(endpoints)}")
    print(f"Used endpoints: {len(used_endpoints)}")
    print(f"Unused endpoints: {len(unused_endpoints)}")


if __name__ == "__main__":
    main()


API Endpoints Analysis:

Used Endpoints:
✓ GET / (in ./backend\webui_backend\routers\users.py)
✓ POST /add (in ./backend\webui_backend\routers\pipelines.py)
✓ POST /admin/config (in ./backend\webui_backend\routers\auths.py)
✓ GET /admin/details (in ./backend\webui_backend\routers\auths.py)
✓ DELETE /all (in ./backend\webui_backend\routers\files.py)
✓ GET /all/archived (in ./backend\webui_backend\routers\chats.py)
✓ GET /all/db (in ./backend\webui_backend\routers\chats.py)
✓ GET /all/tags (in ./backend\webui_backend\routers\chats.py)
✓ GET /api/changelog (in ./backend\webui_backend\main.py)
✓ POST /api/chat (in ./backend\webui_backend\routers\ollama.py)
✓ POST /api/chat/completed (in ./backend\webui_backend\main.py)
✓ POST /api/chat/completions (in ./backend\webui_backend\main.py)
✓ GET /api/config (in ./backend\webui_backend\main.py)
✓ POST /api/copy (in ./backend\webui_backend\routers\ollama.py)
✓ POST /api/create (in ./backend\webui_backend\routers\ollama.py)
✓ DELETE /api/delete (i